# Задачі для теоретичного заняття дванадцятого дня

Задачі теоретичного блоку: продемонструвати на практичному прикладі власні вийнятки і контейнери, створені за допомогою наслідування (UserList, UserDict, UserString), @dataclass, Enum.
Для самостійного опрацювання і спроб запускати всі існуючі задачі не заважаючи іншим студентам курсу потрібно здійснити наступні кроки:

1. Натиснути кнопку "Файл" в лівому верхньому куті
2. Вибрати пункт "Зберегти копію на диску"
3. У вікні з вашою копією у правому верхньому куті натиснути "Підключитись"

Можете запускати всі задачі, які забажаєте і писати власні програми вивчаючи функціонал цього середовища


## [Patterns](https://refactoring.guru/design-patterns)

## Контейнери, створені за допомогою успадкування (UserList, UserDict, UserString)

### Доступ до елементів контейнера

__Магічні методи:__
```python
__len__(self) # Довжина

__getitem__(self, key) # Синатксис доступу за ключем (container[key])
   
__setitem__(self, key, value) # Синтаксис зміни за ключем (container[key] = new_value)

__delitem__(self, key) # Видалення елемента (del container[key])

__iter__(self) # Метод, який викликається функцією iter() або коли використовується `for ... in container:`

__reversed__(self) # Повертає container у зворотному порядку (якщо контейнер упорядкований)

__contains__(self, item) # Відповідає за синтаксис `some_val in container`

__missing__(self, key) # Відповідає за поведінку під час спроби звернутися до неіснуючого елемента
```

#### UserList

In [ ]:
from collections import UserList
from random import randint

class ExperimentsResults(UserList):

    def positive_values(self):
        return list(filter(lambda x: x >= 0, self.data))

    def negative_values(self):
        return list(filter(lambda x: x < 0, self.data))


result = ExperimentsResults()

for _ in range(20):
    result.append(randint(-10, 10))

print(result)
print(result.positive_values())
print(result.negative_values())
print(result.data)

#### UserDict

Створити словник, який дає змогу знаходити/змінювати значення за
ключем і ключ за значенням. Передбачається, що значення не повторюються

In [ ]:
from collections import UserDict


class DualDict(UserDict):
    def __init__(self, initial=None):
        if initial is None:
            self.from_value = dict()
            self.data = dict()
        else:
            self.data = initial
            self.from_value = {val: key for key, val in initial.items()}

    def __setitem__(self, key, value):
        self.data[key] = value
        if value in self.from_value:
            old = self.from_value[value]
            self.from_value.pop(old)
        self.from_value[value] = key

    def get_by_value(self, value):
        return self.from_value[value]

    def set_by_value(self, value, key):
        old_key = self.from_value[value]
        self.data.pop(old_key)
        self.data[key] = value
        self.from_value[value] = key

dual = DualDict({1:2, 3:4})
print(dual)
print(dual.get_by_value(2) == 1)
dual.set_by_value(4, 8)
print(dual)




#### UserString

In [ ]:
from collections import UserString

template = [
    "You can achieve anything. All it takes is a little effort and some books.",
    "This smartphone is the real deal. Big, bright screen, powerful processor, all in a small gadget.",
    "Collecting infinity stones is easy if you're a born hero.",
    "Mastering layout is easy. Pick up a new book and put all the exercises into practice.",
    "It's not hard to fight procrastination. Just take action. In small steps.",
    "Programming isn't as hard as they say.",
    "Simple daily exercises will help you succeed.",
]

class Comments(UserString):
    def get_limit_comment(self, max_len=15):
        return f"{self.data[:max_len-3]}..."

comments = [Comments(el) for el in template]


for comment in comments:
    # print(comment.get_limit_comment())
    print(comment.get_limit_comment(35))

## Власні винятки

In [ ]:
class LessThanZero(Exception):
    pass
    # def __init__(self, value):
    #     self.value = value
    #
    # def __str__(self):
    #     return repr(self.value)

def func(n):
    if n < 0:
        raise LessThanZero(f'value {n} < 0')
    print(n)

func(1)

## Dataclass

In [ ]:
# Dont use dataclass
class LessThanZero:
    def __init__(self):
        self.text()

    def text(self):
        print("You launch LessThanZero class")

test = LessThanZero()

## [`__slots__`](https://stackoverflow.com/questions/472000/usage-of-slots)



## Enum

In [ ]:
from enum import Enum

class Semaphore(Enum):
    RED = 1
    YELLOW = 2
    GREEN = 3

def handle_semaphore(light):
    match light:
        case Semaphore.RED:
            print("You must stop")
        case Semaphore.YELLOW:
            print("Light might be changed to red, be careful")
        case Semaphore.GREEN:
            print("You can continue")

handle_semaphore(Semaphore.RED)
handle_semaphore(Semaphore.YELLOW)
handle_semaphore(Semaphore.GREEN)

In [ ]:
from enum import Enum
from dataclasses import dataclass
from collections import UserList

class UserRole(Enum):
    ADMIN = 'Admin'
    MODERATOR = 'Moderator'
    USER = 'User'


class UnauthorizedAccessError(Exception):
    pass

@dataclass
class Country:
    name: str
    code: str

@dataclass
class User:
    username: str
    email: str
    role: UserRole
    country: Country

class RestrictedUserList(UserList):
    def __init__(self, *args, allowed_roles=None, **kwargs):
        self.allowed_roles = allowed_roles or [UserRole.ADMIN]
        super().__init__(*args, **kwargs)

    def append(self, user) -> None:
        if user.role not in self.allowed_roles:
            raise UnauthorizedAccessError("You don't have permission to add this user")
        super().append(user)

    def extend(self, users) -> None:
        for user in users:
            self.append(user)

    def remove(self, user) -> None:
        if user.role not in self.allowed_roles:
            raise UnauthorizedAccessError("You don't have permission to remove this user")
        super().remove(user)

if __name__ == '__main__':
    usa = Country(name='United States', code="US")
    uk = Country(name='United Kingdom', code="UK")

    admin = User(username='admin', email='admin@admin.com', role=UserRole.ADMIN, country=usa)
    moderator = User(username='moderator', email='moderator@moderator.com', role=UserRole.MODERATOR, country=uk)
    user = User(username='user', email='test@test.com', role=UserRole.USER, country=usa)

    restricted_users = RestrictedUserList(allowed_roles=[UserRole.ADMIN, UserRole.MODERATOR])
    restricted_users.extend([admin, moderator])

    try:
        restricted_users.append(user)
    except UnauthorizedAccessError as e:
        print(e)